# 惯例的helloworld

首先我们应该对Cpp程序有个大概的概念.

cpp程序需要编译.其源文件(source files)一般以`.cpp`,`.cxx`或者`.cc`为后缀.源文件的作用是定义程序的实现,也就是人能看得懂的写给机器真正执行的文件.

一般微软规范中使用`.cpp`而`linux`中常用`.cc`个人认为`.cpp`更合理.不过其实随便.cpp的编译需要编译器,最常见的就是`GNU g++`命令编译.




这两种形式的输出都为int类型.如果返回0,则代表程序正常退出.返回其它数字的含义则由系统决定.通常,返回非零代表程序异常退出.

下面是名为`helloworld.cpp`的源文件.这个源文件有如下几个部分:

+ 引入头文件,c和c++中有头文件这种设定,一般由`.hpp`或者`.h`为后缀.头文件是编译器处理的文件而不是程序执行的文件,它一般用于:

    + 声明源文件的接口
    + 申明模板
    + 申明一些常量,和自定义类型
    
    头文件的引入就是在引入位置直接复制头文件中的内容,因此也有用来定义要引入的文本的情况.这种情况其实并不是头文件,因此我们在这种情况下使用`.inc`为后缀
   
+ 入口函数main函数.

    所有cpp程序都要有一个入口函数--main函数,它可以是两种形式

    + `int main(int argc, char *argv[])`其中输入:

        + argc: 参数的个数,不给main()函数传递参数时默认值为1，即至少有一个参数为该可执行文件的文件名(含目录).
        + argv: 为指针数组,分别指向各个字符串参数的首地址,其中argv[0]存储的是可执行文件的文件名的首地址.

    + `int main()`


In [4]:
%%writefile codes/C0_helloworld/section1/src/helloworld.cpp
#include <stdio.h>

int main(){
    printf("hello world");
    return 0;
}

Writing codes/C0_helloworld/section1/src/helloworld.cpp


In [5]:
!g++ -o codes/C0_helloworld/section1/bin/helloworld \
codes/C0_helloworld/section1/src/helloworld.cpp

In [6]:
!./codes/C0_helloworld/section1/bin/helloworld

'.' 不是内部或外部命令，也不是可运行的程序
或批处理文件。


## 多文件模块化编程

一个项目一般不会只有一个源文件,毕竟只有一个文件的话代码很不好管理.人能照顾到的代码行数极限是2000行,像我们pythoner这样被惯坏了的估计200行就够呛了.C/C++是支持多文件编译的,但你得告诉编译器你定义了哪些方法,类型什么的要让其他模块也可以使用的,也就是所谓的"申明".这就得使用头文件了.下面的头文件就申明了`string hello_name(string)`这个函数,他告诉编译器有个输入参数为string类型,输出也是string类型的名为helo_name的函数可以在与这个头文件同名的源文件中定义具体是如何完成任务的,也就是"实现".同时也可以在其他的头文件,源文件中通过`#include`这个头文件来使用这个函数.


头文件的引入语句`#include`语法有两种

+ `#include <xxx>`表示从标准库中引用
+ `#include "xxx"`表示从自定义的地址中引用,xxx可以是相对地址

在linux下编译器默认从`/usr/include`,`/usr/local/include`引入头文件.这些地方都找不到的话就会在源文件所在的同一目录下找头文件. 要指定额外的查找目录,可以在编译时使用`-I path`指定目录.

在头文件中我们使用
```cpp
#ifndef HELLOWORLD_H
#define HELLOWORLD_H
要引入的头文件
要定义的类型
要声明的函数
.
.
.
#endif
```
这样的宏来包裹定义的内容,这种写法是为了避免头文件的递归引入.像在python中我们要通过模块结构来避免模块的递归引入,C/C++的这种方式虽然看起来比较原始,但反而更加高效.


多源文件编译一般只要将需要用到的源文件一起编译即可,不过为了方便我们一般会将用到的源文件放在同一目录下然后使用通配符`*.cpp`全部指定

In [7]:
%%writefile codes/C0_helloworld/section2/include/helloworld.hpp
#ifndef HELLOWORLD_H
#define HELLOWORLD_H
#include <string>
using std::string;
string hello_name(string);
#endif

Writing codes/C0_helloworld/section2/include/helloworld.hpp


In [8]:
%%writefile codes/C0_helloworld/section2/src/helloworld.cpp
#include <string>
#include "helloworld.hpp"
using std::string;
string hello_name(string name){
    return "hello world "+name;
}

Writing codes/C0_helloworld/section2/src/helloworld.cpp


In [9]:
%%writefile codes/C0_helloworld/section2/src/main.cpp
#include <iostream>
#include "helloworld.hpp"
using std::cout;
using std::endl;
int main(){
    cout << hello_name("Cpp!") <<endl;
    return 0;
}

Writing codes/C0_helloworld/section2/src/main.cpp


In [11]:
!g++ -I codes/C0_helloworld/section2/include \
-o codes/C0_helloworld/section2/bin/helloworld \
codes/C0_helloworld/section2/src/*.cpp

In [12]:
!codes/C0_helloworld/section2/bin/helloworld

'codes' 不是内部或外部命令，也不是可运行的程序
或批处理文件。


## 使用命名空间区分模块

命名空间是C++特有的,C语言中因为没有命名空间,所以要避免重名就很麻烦.命名空间的用处就是区分不用来源的同名变量.有点类似一个定语,比如三班有小明四班也有小明,在各自班,老师喊小明都不会有歧义,但如果全校广播就会造成混乱.所以全校广播的话就会说是找三班的小明或者找四班的小明.在c中往往会出现那种比如`mylib_mypack_func`这样"自带命名空间"的变量/函数命名方式,其实命名空间本质上就是这种行为的规范化扩展

### 创建命名空间

我们使用

```C++
namespace <name> {
    xxx
}
```

来创建命名空间,这个空间内的所有变量常量函数类都将不能直接访问,而要借助命名空间才能访问

### 使用命名空间

使用命名空间又两种方式:

1. 直接使用
    
    `std::cout`就是典型,其中`<namespace>::<object>`代表所属关系,可以和python中的这种引用模块的方式做类比:
    
    ```python
    import numpy
    
    numpy.add(1,2)
    ```
    
    `::`符号就类似python模块引用中的`.`,起到表明从属关系的作用.
    
2. 将命名空间中的对象引入当前空间

    使用`using <namespace>::<object>;`就可以将命名空间中的对象引入当前的变量空间,这就有点类似python中的`from numpy import add`,这样的坏处就是容易污染当前变量空间

    
### 为命名空间创建别名

依然是使用`namespace`关键字,使用如下表达式:`namespace <new_name>=<old_name>;`这样就可以使用`<new_name>`代替`<old_name>`了,不过要注意,这只是别名,并不是替换.

In [13]:
%%writefile codes/C0_helloworld/section3/include/helloworld.hpp
#ifndef HELLOWORLD_H
#define HELLOWORLD_H
#include <string>
using std::string;
namespace Hello{
    string hello_name(string);
}
#endif

Writing codes/C0_helloworld/section3/include/helloworld.hpp


In [14]:
%%writefile codes/C0_helloworld/section3/src/helloworld.cpp
#include <string>
#include "helloworld.hpp"
using std::string;  
string Hello::hello_name(string name){
    return "hello world "+name;
}

Writing codes/C0_helloworld/section3/src/helloworld.cpp


In [15]:
%%writefile codes/C0_helloworld/section3/src/main.cpp
#include <iostream>
#include "helloworld.hpp"
using std::cout;
using std::endl;
using Hello::hello_name;
int main(){
    cout << hello_name("Cpp!") <<endl;
    return 0;
}

Writing codes/C0_helloworld/section3/src/main.cpp


In [17]:
!g++ -I codes/C0_helloworld/section3/include \
-o codes/C0_helloworld/section3/bin/helloworld \
codes/C0_helloworld/section3/src/*.cpp

In [13]:
!codes/C0_helloworld/section3/bin/helloworld

hello world Cpp!


C++的程序结构大致如此.C++中一般会使用namespace来将代码封装为模块.但如果不用,像c语言中一样通过变量名来区分也没问题.